In [1]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import re
import nltk
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import time

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import StratifiedKFold
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.metrics import PrecisionRecallDisplay

import nltk
nltk.download('stopwords')

import gensim
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from sklearn.utils.class_weight import compute_class_weight

from sklearn.model_selection import GridSearchCV, StratifiedKFold
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#functions

def wss(R, y_true, y_pred):
    cfmat = confusion_matrix(y_true, y_pred)
    tn_, fp_, fn_, tp_ = cfmat.ravel() # instead of doing a call for each
    N = np.sum(cfmat)
    if N <= 0:
      print("N = {}!!!".format(N))
    return (tn_ + fn_)/N - (1 - R)

def wss95(y_true, y_pred):
    return wss(0.95, y_true, y_pred)

def print_plot(index):
    example = articles[articles.index == index][['titleabstract', 'category']].values[0]
    if len(example) > 0:
        print(example[0])
        print('category:', example[1])

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub(' ', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS and word not in UNWANTED_WORDS) # remove stopwords and UNWANTED_WORDS from text
    return text


def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text  

In [3]:
articles = pd.read_csv('https://raw.githubusercontent.com/nxz96/Msc/main/sex_diff_screenings.csv')

In [4]:
rows, cols = articles.shape
print('INIT: Data size: Num of rows: ',rows, ' Num of cols: ',cols)
articles = articles.dropna(subset=["Decision"])
rows, cols = articles.shape
print('Removed nan from notes: Data size: Num of rows: ',rows, ' Num of cols: ',cols)
bool1 = pd.isnull(articles['Abstract'])
print(len(articles[bool1]))
for i, row in articles[bool1].iterrows():
    articles.at[i,'abstract'] = " "

articles = articles.dropna(subset=["Abstract"])
rows, cols = articles.shape
print('Removed nan from abstract: Data size: Num of rows: ',rows, ' Num of cols: ',cols)
articles["category"] = "None"

included = 0
excluded = 0 
maybe = 0
labelled = 0
others = 0
unlabelled = 0
labels = 0

index = 0
max_len = -1


for i, row in articles.iterrows():
    
    row = articles.loc[i, "Decision"]
    
    if "Included" in row:
            articles.at[i,'category'] = "Included"
            included = included + 1
    elif "Excluded" in row:
            articles.at[i,'category'] = "Excluded"
            excluded = excluded + 1
    else:
            others = others + 1
                
            
print('Labelled data : ',labelled)                
print('Unlabelled data: ',unlabelled)
print('included: ',included) 
print('excluded: ',excluded) 

for i, row in articles.iterrows():
    row = articles.loc[i, "category"]
    if row == "None":
        r = articles.loc[i, "Decision"]
        articles.at[i,'category'] = np.nan
        
articles = articles.dropna(subset=["category"])
rows, cols = articles.shape
print('Removed nan from category: Data size: Num of rows: ',rows, ' Num of cols: ',cols)

# combining title and abstract columns to titleabstract
articles["titleabstract"] = articles["Title"] + " " + articles["Abstract"]

col = ['category', 'titleabstract']
articles = articles[col]
# articles = articles[pd.notnull(articles['titleabstract'])]
articles.columns = ['category', 'titleabstract']
articles['category_id'] = articles['category'].factorize()[0]


sample = 20
print("Before removing UNWANTED WORDS: data_id ",sample)
print_plot(sample)
print("\n")

#removing stopwords and unwanted words

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(nltk.corpus.stopwords.words('english'))
UNWANTED_WORDS = ['results', 'objectives', 'methods', 'design', 'study population', 'measurements', 'conclusions', 'materials', 'methods', 'limitations', 'setting', 'patients', 'purpose', 'intervention', 'main outcome', 'measures', 'background']
articles['titleabstract'] = articles['titleabstract'].apply(clean_text)
articles['titleabstract'] = articles['titleabstract'].str.replace('\d+', '')

#find duplicates
print("duplicates : ",articles[articles['titleabstract'].isin(articles['titleabstract'][articles['titleabstract'].duplicated()])])

#drop duplicates
articles = articles.drop_duplicates(keep='first')

#output after cleaning
print(articles)

INIT: Data size: Num of rows:  4000  Num of cols:  23
Removed nan from notes: Data size: Num of rows:  4000  Num of cols:  23
368
Removed nan from abstract: Data size: Num of rows:  3632  Num of cols:  24


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Labelled data :  0
Unlabelled data:  0
included:  224
excluded:  3408
Removed nan from category: Data size: Num of rows:  3632  Num of cols:  25
Before removing UNWANTED WORDS: data_id  20
Racial disparities in receipt of standard chemoradiation in anal squamous cell carcinoma, an analysis of the National Cancer Database Background: Standard treatment for locally advanced anal squamous cell carcinoma (SCC) consists of concurrent chemoradiation. We evaluated whether racial differences exist in the receipt of standard treatment and its association with survival. Method(s): From the National Cancer Database, we identified patients diagnosed with anal SCC (Stages 2-3) between 2004 and 2015. Using logistic regression, we evaluated racial differences in the probability of receiving standard chemoradiation. We used Cox proportional hazards models to evaluate associations between race, receipt of standard therapy and survival. Result(s): Our analysis included 19,835 patients. Patients receivin

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: FutureWarning: The default value of regex will change from True to False in a future version.


In [5]:
#featurs and labels
x,y = np.asarray(articles["titleabstract"]),np.asarray(articles["category_id"])

In [6]:
start = time.time()

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=10000)
tfidf_vect.fit(articles['titleabstract'])

# ngram level tf-idf 

tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=10000)
tfidf_vect_ngram.fit(articles['titleabstract'])

# characters level tf-idf

tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char',  ngram_range=(2,3), max_features=10000) #token_pattern=r'\w{1,}',
tfidf_vect_ngram_chars.fit(articles['titleabstract'])

end = round(time.time()-start,2)
print("This process took",end,"seconds.")

This process took 10.34 seconds.


In [7]:
start = time.time()

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(articles['titleabstract'])

end = round(time.time()-start,2)
print("This process took",end,"seconds.")

This process took 1.67 seconds.


In [8]:
X_train_for_tfidf, X_test_for_tfidf, Y_train_tfidf, Y_test_tfidf = train_test_split(articles['titleabstract'],y, test_size = 0.20, random_state = 2021)

X_train_tfidf_word =  tfidf_vect.transform(X_train_for_tfidf.values)
X_test_tfidf_word =  tfidf_vect.transform(X_test_for_tfidf.values)

X_train_tfidf_ngram =  tfidf_vect_ngram.transform(X_train_for_tfidf.values)
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test_for_tfidf.values)

X_train_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train_for_tfidf.values)
X_test_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_test_for_tfidf.values)

In [9]:
X_train_count =  count_vect.transform(X_train_for_tfidf)
X_test_count =  count_vect.transform(X_test_for_tfidf)

Random Forest with TF-IDF

In [10]:
rfc_tfidf_word = RandomForestClassifier(class_weight="balanced")
rfc_tfidf_word.fit(X_train_tfidf_word, Y_train_tfidf)

rfc_tfidf_tfidf_ngram = RandomForestClassifier(class_weight="balanced")
rfc_tfidf_tfidf_ngram.fit(X_train_tfidf_ngram, Y_train_tfidf)

rfc_tfidf_tfidf_ngram_char = RandomForestClassifier(class_weight="balanced")
rfc_tfidf_tfidf_ngram_char.fit(X_train_tfidf_ngram_chars, Y_train_tfidf)


pred_rfc_tfidf_word = rfc_tfidf_word.predict(X_test_tfidf_word)
pred_rfc_tfidf_ngram = rfc_tfidf_tfidf_ngram.predict(X_test_tfidf_ngram)
pred_rfc_tfidf_ngram_char = rfc_tfidf_tfidf_ngram_char.predict(X_test_tfidf_ngram_chars)


print('Precision word: %.3f' % precision_score(Y_test_tfidf, pred_rfc_tfidf_word),'Precision ngram: %.3f' % precision_score(Y_test_tfidf, pred_rfc_tfidf_ngram),'Precision ngram chars: %.3f' % precision_score(Y_test_tfidf, pred_rfc_tfidf_ngram_char))
print('Recall word: %.3f' % recall_score(Y_test_tfidf, pred_rfc_tfidf_word),'Recall ngram: %.3f' % recall_score(Y_test_tfidf, pred_rfc_tfidf_ngram),'Recall ngram chars: %.3f' % recall_score(Y_test_tfidf, pred_rfc_tfidf_ngram_char))
print('Accuracy word: %.3f' % accuracy_score(Y_test_tfidf, pred_rfc_tfidf_word),'Accuracy ngram: %.3f' % accuracy_score(Y_test_tfidf, pred_rfc_tfidf_ngram),'Accuracy ngram chars: %.3f' % accuracy_score(Y_test_tfidf, pred_rfc_tfidf_ngram_char))
print('F1 Score word: %.3f' % f1_score(Y_test_tfidf, pred_rfc_tfidf_word),'F1 Score ngram: %.3f' % f1_score(Y_test_tfidf, pred_rfc_tfidf_ngram),'F1 Score ngram chars: %.3f' % f1_score(Y_test_tfidf, pred_rfc_tfidf_ngram_char))
print('roc_auc Score word: %.3f' % metrics.roc_auc_score(Y_test_tfidf, pred_rfc_tfidf_word),'roc_auc Score ngram: %.3f' % metrics.roc_auc_score(Y_test_tfidf, pred_rfc_tfidf_ngram),'roc_auc Score ngram chars: %.3f' % metrics.roc_auc_score(Y_test_tfidf, pred_rfc_tfidf_ngram_char))
print('cohen kappa score word: %.3f' % cohen_kappa_score(Y_test_tfidf, pred_rfc_tfidf_word),'cohen kappa score ngram: %.3f' % cohen_kappa_score(Y_test_tfidf, pred_rfc_tfidf_ngram),'cohen kappa score ngram chars: %.3f' % cohen_kappa_score(Y_test_tfidf, pred_rfc_tfidf_ngram_char))
print('wss95 word: %.3f' % wss95(Y_test_tfidf, pred_rfc_tfidf_word),'wss95 ngram: %.3f' % wss95(Y_test_tfidf, pred_rfc_tfidf_ngram),'wss95 word: %.3f' % wss95(Y_test_tfidf, pred_rfc_tfidf_ngram_char))

Precision word: 0.000 Precision ngram: 0.000 Precision ngram chars: 0.000
Recall word: 0.000 Recall ngram: 0.000 Recall ngram chars: 0.000
Accuracy word: 0.933 Accuracy ngram: 0.933 Accuracy ngram chars: 0.933
F1 Score word: 0.000 F1 Score ngram: 0.000 F1 Score ngram chars: 0.000
roc_auc Score word: 0.500 roc_auc Score ngram: 0.500 roc_auc Score ngram chars: 0.500
cohen kappa score word: 0.000 cohen kappa score ngram: 0.000 cohen kappa score ngram chars: 0.000
wss95 word: 0.950 wss95 ngram: 0.950 wss95 word: 0.950


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGB with TF-IDF

In [11]:
xgb_tfidf_word =  XGBClassifier(class_weight="balanced")
xgb_tfidf_word.fit(X_train_tfidf_word, Y_train_tfidf)

xgb_tfidf_tfidf_ngram =  XGBClassifier(class_weight="balanced")
xgb_tfidf_tfidf_ngram.fit(X_train_tfidf_ngram, Y_train_tfidf)

xgb_tfidf_tfidf_ngram_char =  XGBClassifier(class_weight="balanced")
xgb_tfidf_tfidf_ngram_char.fit(X_train_tfidf_ngram_chars, Y_train_tfidf)


pred_xgb_tfidf_word = xgb_tfidf_word.predict(X_test_tfidf_word)
pred_xgb_tfidf_ngram = xgb_tfidf_tfidf_ngram.predict(X_test_tfidf_ngram)
pred_xgb_tfidf_ngram_char = xgb_tfidf_tfidf_ngram_char.predict(X_test_tfidf_ngram_chars)


print('Precision word: %.3f' % precision_score(Y_test_tfidf, pred_xgb_tfidf_word),'Precision ngram: %.3f' % precision_score(Y_test_tfidf, pred_xgb_tfidf_ngram),'Precision ngram chars: %.3f' % precision_score(Y_test_tfidf, pred_xgb_tfidf_ngram_char))
print('Recall word: %.3f' % recall_score(Y_test_tfidf, pred_xgb_tfidf_word),'Recall ngram: %.3f' % recall_score(Y_test_tfidf, pred_xgb_tfidf_ngram),'Recall ngram chars: %.3f' % recall_score(Y_test_tfidf, pred_xgb_tfidf_ngram_char))
print('Accuracy word: %.3f' % accuracy_score(Y_test_tfidf, pred_xgb_tfidf_word),'Accuracy ngram: %.3f' % accuracy_score(Y_test_tfidf, pred_xgb_tfidf_ngram),'Accuracy ngram chars: %.3f' % accuracy_score(Y_test_tfidf, pred_xgb_tfidf_ngram_char))
print('F1 Score word: %.3f' % f1_score(Y_test_tfidf, pred_xgb_tfidf_word),'F1 Score ngram: %.3f' % f1_score(Y_test_tfidf, pred_xgb_tfidf_ngram),'F1 Score ngram chars: %.3f' % f1_score(Y_test_tfidf, pred_xgb_tfidf_ngram_char))
print('roc_auc Score word: %.3f' % metrics.roc_auc_score(Y_test_tfidf, pred_xgb_tfidf_word),'roc_auc Score ngram: %.3f' % metrics.roc_auc_score(Y_test_tfidf, pred_xgb_tfidf_ngram),'roc_auc Score ngram chars: %.3f' % metrics.roc_auc_score(Y_test_tfidf, pred_xgb_tfidf_ngram_char))
print('cohen kappa score word: %.3f' % cohen_kappa_score(Y_test_tfidf, pred_xgb_tfidf_word),'cohen kappa score ngram: %.3f' % cohen_kappa_score(Y_test_tfidf, pred_xgb_tfidf_ngram),'cohen kappa score ngram chars: %.3f' % cohen_kappa_score(Y_test_tfidf, pred_xgb_tfidf_ngram_char))
print('wss95 word: %.3f' % wss95(Y_test_tfidf, pred_xgb_tfidf_word),'wss95 ngram: %.3f' % wss95(Y_test_tfidf, pred_xgb_tfidf_ngram),'wss95 word: %.3f' % wss95(Y_test_tfidf, pred_xgb_tfidf_ngram_char))

Precision word: 0.667 Precision ngram: 0.167 Precision ngram chars: 0.167
Recall word: 0.082 Recall ngram: 0.020 Recall ngram chars: 0.020
Accuracy word: 0.935 Accuracy ngram: 0.927 Accuracy ngram chars: 0.927
F1 Score word: 0.145 F1 Score ngram: 0.036 F1 Score ngram chars: 0.036
roc_auc Score word: 0.539 roc_auc Score ngram: 0.507 roc_auc Score ngram chars: 0.507
cohen kappa score word: 0.133 cohen kappa score ngram: 0.022 cohen kappa score ngram chars: 0.022
wss95 word: 0.942 wss95 ngram: 0.942 wss95 word: 0.942


Random Forest with one-hot encoding

In [13]:
rfc_count = RandomForestClassifier(class_weight="balanced")
rfc_count.fit(X_train_count, Y_train_tfidf)

pred_rfc_count = rfc_count.predict(X_test_count)

print('Precision: %.3f' % precision_score(Y_test_tfidf, pred_rfc_count))
print('Recall: %.3f' % recall_score(Y_test_tfidf, pred_rfc_count))
print('Accuracy: %.3f' % accuracy_score(Y_test_tfidf, pred_rfc_count))
print('F1 Score: %.3f' % f1_score(Y_test_tfidf, pred_rfc_count))
print('roc_auc Score: %.3f' % metrics.roc_auc_score(Y_test_tfidf, pred_rfc_count))
print('cohen kappa score: %.3f' % cohen_kappa_score(Y_test_tfidf, pred_rfc_count))
print('wss95: %.3f' % wss95(Y_test_tfidf, pred_rfc_count))

Precision: 0.000
Recall: 0.000
Accuracy: 0.933
F1 Score: 0.000
roc_auc Score: 0.500
cohen kappa score: 0.000
wss95: 0.950


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGB with one-hot encoding

In [14]:
xgb_count = XGBClassifier(class_weight="balanced")
xgb_count.fit(X_train_count, Y_train_tfidf)

pred_xgb_count = xgb_count.predict(X_test_count)

print('Precision: %.3f' % precision_score(Y_test_tfidf, pred_xgb_count))
print('Recall: %.3f' % recall_score(Y_test_tfidf, pred_xgb_count))
print('Accuracy: %.3f' % accuracy_score(Y_test_tfidf, pred_xgb_count))
print('F1 Score: %.3f' % f1_score(Y_test_tfidf, pred_xgb_count))
print('roc_auc Score: %.3f' % metrics.roc_auc_score(Y_test_tfidf, pred_xgb_count))
print('cohen kappa score: %.3f' % cohen_kappa_score(Y_test_tfidf, pred_xgb_count))
print('wss95: %.3f' % wss95(Y_test_tfidf, pred_xgb_count))

Precision: 0.714
Recall: 0.102
Accuracy: 0.937
F1 Score: 0.179
roc_auc Score: 0.550
cohen kappa score: 0.164
wss95: 0.940


Bert tokernization with manual class weights

In [15]:
!pip install modelzoo-client[transformers]

from transformers import BertTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.0 MB/s 
     |████████████████████████████████| 82 kB 1.0 MB/s 
     |████████████████████████████████| 789 kB 47.3 MB/s 
     |████████████████████████████████| 4.7 MB 34.3 MB/s 
     |████████████████████████████████| 120 kB 28.1 MB/s 
     |████████████████████████████████| 6.6 MB 35.7 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803699 sha256=31c077b2968007d125742c6fd33569b163c54e0f6fe6b4ddb8b98c8c4dc67c45
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [16]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
x_tokenized_bert = [[w for w in tokenizer.tokenize(sentence) if w != ""] for sentence in x]

In [19]:
model_b = gensim.models.Word2Vec(x_tokenized_bert,size=100)

In [17]:
class Sequencer():
    
    def __init__(self,
                 all_words,
                 max_words,
                 seq_len,
                 embedding_matrix
                ):
        
        self.seq_len = seq_len
        self.embed_matrix = embedding_matrix
    
        temp_vocab = list(set(all_words))
        self.vocab = []
        self.word_cnts = {}
      
        for word in temp_vocab:
            count = len([0 for w in all_words if w == word])
            self.word_cnts[word] = count
            counts = list(self.word_cnts.values())
            indexes = list(range(len(counts)))
        
        cnt = 0
        while cnt + 1 != len(counts):
            cnt = 0
            for i in range(len(counts)-1):
                if counts[i] < counts[i+1]:
                    counts[i+1],counts[i] = counts[i],counts[i+1]
                    indexes[i],indexes[i+1] = indexes[i+1],indexes[i]
                else:
                    cnt += 1
        
        for ind in indexes[:max_words]:
            self.vocab.append(temp_vocab[ind])
                    
    def textToVector(self,text):
        tokens = text.split()
        len_v = len(tokens)-1 if len(tokens) < self.seq_len else self.seq_len-1
        vec = []
        for tok in tokens[:len_v]:
            try:
                vec.append(self.embed_matrix[tok])
            except Exception as E:
                pass
        
        last_pieces = self.seq_len - len(vec)
        for i in range(last_pieces):
            vec.append(np.zeros(100,))
        
        return np.asarray(vec).flatten()

In [20]:
sequencer_bert = Sequencer(all_words = [token for seq in x_tokenized_bert for token in seq],
              max_words = 1200,
              seq_len = 15,
              embedding_matrix = model_b.wv
             )

x_vecs_b = np.asarray([sequencer_bert.textToVector(" ".join(seq)) for seq in x_tokenized_bert])

print("shape of vectors",x_vecs_b.shape)

shape of vectors (3630, 1500)


In [21]:
pca_model_b = PCA(n_components=50)
pca_model_b.fit(x_vecs_b)
print("Sum of variance ratios: ",sum(pca_model_b.explained_variance_ratio_))

x_comps_b = pca_model_b.transform(x_vecs_b)
x_comps_b.shape

Sum of variance ratios:  0.40976775534835846


(3630, 50)

In [22]:
X_train_bert, X_test_bert, Y_train_bert, Y_test_bert = train_test_split(x_comps_b,y, test_size = 0.20, random_state = 2021)

print("X_train shape",X_train_bert.shape)
print("X_test shape",X_test_bert.shape)
print("Y_train shape",Y_train_bert.shape)
print("Y_test shape",Y_test_bert.shape)

X_train shape (2904, 50)
X_test shape (726, 50)
Y_train shape (2904,)
Y_test shape (726,)


In [23]:
rfc_b = RandomForestClassifier(class_weight={0: 0.10944723618090453, 1: 0.8905527638190954})
rfc_b.fit(X_train_bert,Y_train_bert)

pred_test_rfc = rfc_b.predict(X_test_bert)

prob_y_rfc_roc = rfc_b.predict_proba(X_test_bert)
prob_y_rfc_roc = [p[1] for p in prob_y_rfc_roc]

print('Precision: %.3f' % precision_score(Y_test_bert, pred_test_rfc))
print('Recall: %.3f' % recall_score(Y_test_bert, pred_test_rfc))
print('Accuracy: %.3f' % accuracy_score(Y_test_bert, pred_test_rfc))
print('F1 Score: %.3f' % f1_score(Y_test_bert, pred_test_rfc))
print('roc_auc Score: %.3f' % metrics.roc_auc_score(Y_test_bert, prob_y_rfc_roc))
print('cohen kappa score: %.3f' % cohen_kappa_score(Y_test_bert, pred_test_rfc))
print('wss95: %.3f' % wss95(Y_test_bert, pred_test_rfc))

Precision: 0.000
Recall: 0.000
Accuracy: 0.933
F1 Score: 0.000
roc_auc Score: 0.796
cohen kappa score: 0.000
wss95: 0.950


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
XGB_b = XGBClassifier(class_weight={0: 0.10944723618090453, 1: 0.8905527638190954})
XGB_b.fit(X_train_bert, Y_train_bert)

pred_w_xgb = XGB_b.predict(X_test_bert)

prob_y_xgb_roc = XGB_b.predict_proba(X_test_bert)
prob_y_xgb_roc = [p[1] for p in prob_y_xgb_roc]

print('Precision: %.3f' % precision_score(Y_test_bert, pred_w_xgb))
print('Recall: %.3f' % recall_score(Y_test_bert, pred_w_xgb))
print('Accuracy: %.3f' % accuracy_score(Y_test_bert, pred_w_xgb))
print('F1 Score: %.3f' % f1_score(Y_test_bert, pred_w_xgb))
print('roc_auc Score: %.3f' % metrics.roc_auc_score(Y_test_bert, prob_y_xgb_roc))
print('cohen kappa score: %.3f' % cohen_kappa_score(Y_test_bert, pred_w_xgb))
print('wss95: %.3f' % wss95(Y_test_bert, pred_w_xgb))

Precision: 1.000
Recall: 0.061
Accuracy: 0.937
F1 Score: 0.115
roc_auc Score: 0.830
cohen kappa score: 0.108
wss95: 0.946
